In [1]:
# Before running, make sure you have the necessary libraries installed.
# If not, run this in your environment (e.g., Google Colab, Jupyter Notebook):
!pip install transformers sentence-transformers gradio faiss-cpu requests

# ✅ STEP 1: Imports
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer
from transformers import pipeline
import gradio as gr
import logging
import requests
import time
import json # New import for JSON caching
import os   # New import for file system operations

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', force=True)

# Define the cache file name
CACHE_FILE = 'wikipedia_summary_cache.json'
wikipedia_cache = {}

# Load existing cache if it exists
if os.path.exists(CACHE_FILE):
    try:
        with open(CACHE_FILE, 'r', encoding='utf-8') as f:
            wikipedia_cache = json.load(f)
        logging.info(f"Loaded existing Wikipedia summary cache from {CACHE_FILE}.")
    except json.JSONDecodeError as e:
        logging.warning(f"Error decoding JSON from {CACHE_FILE}: {e}. Starting with an empty cache.")
        wikipedia_cache = {}
else:
    logging.info(f"No existing cache file found at {CACHE_FILE}. A new cache will be created.")


# ✅ STEP 2: Initialize Sentence-BERT Model and FAISS Index
logging.info("Initializing Sentence-BERT model and FAISS index...")
try:
    model = SentenceTransformer("all-MiniLM-L6-v2")  # fast, lightweight
    index = faiss.IndexFlatL2(384)  # for 384-dimensional embeddings (output of all-MiniLM-L6-v2)
    docs = []  # store document chunks here (e.g., country paragraphs)
    logging.info("Model and FAISS index initialized successfully.")
except Exception as e:
    logging.error(f"FATAL ERROR: Could not initialize model or FAISS index. Please check your environment. Error: {e}")
    import sys
    sys.exit(1)

# Function to fetch Wikipedia summary directly using requests with backoff (NOW WITH CACHING)
def fetch_wikipedia_summary_robust(title, sentences=3, max_retries=5, initial_delay=2): # Increased defaults slightly
    """
    Fetches a summary from Wikipedia using its API, with retry logic and exponential backoff
    to handle rate limiting and temporary network issues. Also attempts to resolve
    disambiguation pages by performing a search.
    Includes caching to avoid repeated API calls.
    """
    # Check cache first
    if title in wikipedia_cache:
        logging.info(f"Using cached Wikipedia summary for '{title}'.")
        return wikipedia_cache[title]

    API_URL = "https://en.wikipedia.org/w/api.php"
    params = {
        "action": "query", "format": "json", "titles": title,
        "prop": "extracts", "exintro": True, "explaintext": True, "exsentences": sentences
    }
    current_delay = initial_delay
    summary_extract = None # Initialize to None

    for attempt in range(max_retries):
        try:
            response = requests.get(API_URL, params=params, timeout=10)
            response.raise_for_status()

            data = response.json()
            pages = data.get("query", {}).get("pages", {})

            page_id = next(iter(pages.keys()), None)

            if page_id is None or "-1" in pages:
                logging.warning(f"Wikipedia API: Page '{title}' not found (Attempt {attempt+1}/{max_retries}).")
                summary_extract = None # No page found, so no extract
                break # Exit retry loop, page not found

            page_data = pages.get(page_id, {})
            extract = page_data.get("extract", "").strip()

            if "may refer to:" in extract.lower() or "(disambiguation)" in page_data.get("title", "").lower() or len(extract) < 50:
                logging.warning(f"Wikipedia API: '{title}' is potentially a disambiguation page or too short (Attempt {attempt+1}/{max_retries}). Attempting search...")

                search_params = {
                    "action": "query", "format": "json", "list": "search", "srsearch": title, "srlimit": 1
                }
                search_response = requests.get(API_URL, params=search_params, timeout=10)
                search_response.raise_for_status()
                search_data = search_response.json()

                search_results = search_data.get("query", {}).get("search", [])
                if search_results:
                    new_title = search_results[0].get("title")
                    logging.info(f"Wikipedia API: Found alternative '{new_title}' for '{title}'. Retrying summary.")
                    # Recursively call, but don't cache this intermediate result directly for the original title
                    # The recursive call will handle its own caching for new_title
                    summary_extract = fetch_wikipedia_summary_robust(new_title, sentences, max_retries=1)
                    break # Break retry loop if successful alternative found
                else:
                    logging.warning(f"Wikipedia API: No search results found for '{title}' after disambiguation attempt (Attempt {attempt+1}/{max_retries}).")
                    summary_extract = None # No search result, so no extract
                    break # Exit retry loop

            summary_extract = extract # If we reached here, we have a valid extract
            break # Success, exit retry loop

        except requests.exceptions.HTTPError as http_err:
            if http_err.response.status_code == 429:
                logging.warning(f"Wikipedia API: Rate limit hit for '{title}' (Attempt {attempt+1}/{max_retries}). Retrying in {current_delay:.2f} seconds...")
                time.sleep(current_delay)
                current_delay *= 2
            else:
                logging.error(f"Wikipedia API HTTP Error for '{title}' (Attempt {attempt+1}/{max_retries}): {http_err}")
                summary_extract = None # Error occurred, no extract
                break # Exit retry loop
        except requests.exceptions.ConnectionError as conn_err:
            logging.error(f"Wikipedia API Connection Error for '{title}' (Attempt {attempt+1}/{max_retries}): {conn_err}. Retrying in {current_delay:.2f} seconds...")
            time.sleep(current_delay)
            current_delay *= 2
        except requests.exceptions.Timeout:
            logging.error(f"Wikipedia API Timeout for '{title}' (Attempt {attempt+1}/{max_retries}). Retrying in {current_delay:.2f} seconds...")
            time.sleep(current_delay)
            current_delay *= 2
        except Exception as e:
            logging.error(f"Unexpected error fetching summary for '{title}' (Attempt {attempt+1}/{max_retries}): {e}")
            summary_extract = None # Unexpected error, no extract
            break # Exit retry loop

    if summary_extract is None:
        logging.error(f"Failed to fetch summary for '{title}' after {max_retries} attempts.")
    else:
        # Save to cache if a summary was successfully retrieved
        wikipedia_cache[title] = summary_extract
        # Persist the cache to disk after each successful new fetch
        try:
            with open(CACHE_FILE, 'w', encoding='utf-8') as f:
                json.dump(wikipedia_cache, f, ensure_ascii=False, indent=2)
            logging.info(f"Updated Wikipedia summary cache saved to {CACHE_FILE}.")
        except IOError as e:
            logging.error(f"Error saving cache to {CACHE_FILE}: {e}")

    return summary_extract


# Function to filter out irrelevant chunks from Wikipedia summaries
def filter_wiki_chunk(chunk):
    irrelevant_keywords = [
        "million years ago", "billion years ago", "geological", "archaeological",
        "prehistory", "ancient history", "paleolithic", "mesolithic", "neolithic",
        "bronze age", "iron age", "dynasty", "kingdom", "republic", "empire",
        "world war", "cold war", "revolution", "colonial", "post-colonial",
        "demographics", "gdp", "economic", "political system", "parliament", "president",
        "prime minister", "constitution", "legal system", "military", "armed forces",
        "scientific research", "technological development", "climate change",
        "fauna", "flora", "ecosystem", "biodiversity", # May exclude relevant environmental facts, but for *travel* bot
        "plates", "tectonic", "ethnic", "religious"
    ]
    # Check if any keyword (case-insensitive) is in the chunk
    for keyword in irrelevant_keywords:
        if keyword in chunk.lower():
            return False
    return True

# ✅ STEP 3: Load and Index Knowledge Base
logging.info("Loading and indexing knowledge base from multiple sources...")

countries_to_load = [
    "Japan", "Italy", "France", "United States", "Brazil",
    "Morocco", "Canada", "Thailand", "India", "South Africa",
    "Australia", "Mexico", "China", "Spain", "Germany"
]

# --- Manual Travel Data (for specific questions like "best time to visit", "what to eat", etc.) ---
manual_travel_data = [
    # Japan
    "For Japan, the best time to visit is during spring (March to May) for cherry blossoms. Autumn (September to November) is also an excellent time to visit Japan for its colorful foliage.",
    "When in Japan, popular foods to try include Sushi, Ramen, Tempura, Udon, Soba, and Takoyaki.",
    "Japanese culture emphasizes respect, harmony, and punctuality. Traditional arts include tea ceremony, flower arranging, and Kabuki theater.",
    "The official language of Japan is Japanese. Its currency is the Japanese Yen (JPY).",
    "Top universities in Japan include the University of Tokyo and Kyoto University.",
    "Major tourist places in Japan are Tokyo (Shibuya Crossing, Imperial Palace), Kyoto (Kinkaku-ji, Fushimi Inari-taisha), and Mount Fuji.",

    # Italy
    "The best time to visit Italy is usually spring (April to May) or fall (September to October) to enjoy pleasant weather and fewer crowds.",
    "When in Italy, you must try foods like Pizza, various Pasta dishes, authentic Gelato, Tiramisu, Risotto, and Lasagna.",
    "Italian culture is known for its art, fashion, family values, and passionate way of life. Opera, Roman architecture, and Renaissance art are key elements.",
    "The official language of Italy is Italian. Its currency is the Euro (EUR).",
    "Top universities in Italy include Sapienza University of Rome and the University of Bologna.",
    "Major tourist places in Italy include Rome (Colosseum, Vatican City), Florence (Duomo, Uffizi Gallery), Venice (Grand Canal), and the Amalfi Coast.",

    # France
    "The best time to visit France is spring (April to June) or autumn (September to November) for mild weather and cultural events.",
    "In France, iconic foods to try include Croissants, fresh Baguette with cheese, Coq au Vin, Escargots, delightful Macarons, and Crêpes.",
    "French culture is celebrated for its elegance, haute couture, gastronomy, and rich history in art and philosophy.",
    "The official language of France is French. Its currency is the Euro (EUR).",
    "Top universities in France include PSL Research University Paris and Sorbonne University.",
    "Major tourist places in France are Paris (Eiffel Tower, Louvre Museum), French Riviera, and the Loire Valley castles.",

    # United States
    "The USA is a vast country with varied climates. Generally, spring (April-May) and fall (September-October) offer mild and pleasant weather across many regions, making them ideal times to visit. Summer (June-August) is popular for beaches and national parks, while winter (December-February) is best for snow sports.",
    "Popular American foods include Hamburgers, Hot Dogs, Apple Pie, BBQ ribs, Fried Chicken, and Tacos (Tex-Mex style).",
    "American culture is diverse and influenced by many immigrant groups, emphasizing individualism, innovation, and pop culture.",
    "The official language of the United States is English. Its currency is the United States Dollar (USD).",
    "Top universities in the United States include Harvard University, Stanford University, and MIT.",
    "Major tourist places in the United States are New York City (Statue of Liberty, Times Square), Grand Canyon, Florida's theme parks, and California's national parks.",

    # Brazil
    "The best time to visit Brazil depends on the region, but generally May to September avoids the wet season and peak humidity, and aligns with major festivals.",
    "Traditional Brazilian foods include Feijoada, Pão de Queijo, Açaí, Churrasco (Brazilian BBQ), Coxinha, and Moqueca.",
    "Brazilian culture is vibrant and diverse, known for its Carnival, samba music, football passion, and rich Afro-Brazilian heritage.",
    "The official language of Brazil is Portuguese. Its currency is the Brazilian Real (BRL).",
    "Top universities in Brazil include the University of São Paulo and the University of Campinas.",
    "Major tourist places in Brazil are Rio de Janeiro (Christ the Redeemer, Copacabana Beach), Iguazu Falls, and the Amazon Rainforest.",

    # Morocco
    "The best time to visit Morocco is spring (mid-March to May) or autumn (September to October) for comfortable temperatures.",
    "Moroccan cuisine features Tagine, Couscous, Harira soup, Mint Tea, and Pastilla.",
    "Moroccan culture is a blend of Arab, Berber, and European influences, known for its hospitality, vibrant souks, and intricate architecture.",
    "The official languages of Morocco are Arabic and Berber. Its currency is the Moroccan Dirham (MAD).",
    "Top universities in Morocco include Mohammed V University in Rabat and Cadi Ayyad University in Marrakech.",
    "Major tourist places in Morocco are Marrakech (Djemaa el-Fna, Jardin Majorelle), Fes (tanneries), Chefchaouen (the Blue City), and the Sahara Desert.",

    # Canada
    "The best time to visit Canada is typically summer (June to August) for warm weather and outdoor activities, or winter (December to March) for snow sports.",
    "Canadian foods to try are Poutine, Maple Syrup products, Nanaimo Bars, Tourtière, and BeaverTails.",
    "Canadian culture is known for its politeness, multiculturalism, natural beauty appreciation, and winter sports.",
    "The official languages of Canada are English and French. Its currency is the Canadian Dollar (CAD).",
    "Top universities in Canada include the University of Toronto, McGill University, and the University of British Columbia.",
    "Major tourist places in Canada are Banff National Park, Vancouver, Toronto (CN Tower), and Niagara Falls.",

    # Thailand
    "The best time to visit Thailand is during the cool and dry season (November to February) to avoid extreme heat and heavy rains.",
    "Famous Thai dishes include Pad Thai, Tom Yum Goong, Green Curry, Mango Sticky Rice, and Massaman Curry.",
    "Thai culture is deeply influenced by Buddhism, emphasizing respect, hospitality, and a rich tradition of dance, music, and festivals.",
    "The official language of Thailand is Thai. Its currency is the Thai Baht (THB).",
    "Top universities in Thailand include Chulalongkorn University and Mahidol University.",
    "Major tourist places in Thailand are Bangkok (Grand Palace, Wat Arun), Phuket, Chiang Mai, and the Phi Phi Islands.",

    # India
    "The best time to visit India is generally during the cooler, drier months from October to March, particularly for north and central regions.",
    "Indian cuisine is diverse, with popular dishes like Curry, Biryani, Naan, Dosa, Samosas, and Butter Chicken.",
    "Indian culture is rich in diversity, ancient traditions, spirituality (Hinduism, Buddhism, Sikhism), elaborate festivals, and classical arts.",
    "The official languages of India are Hindi and English. Its currency is the Indian Rupee (INR).",
    "Top universities in India include the Indian Institute of Science (IISc Bangalore) and IIT Delhi.",
    "Major tourist places in India are the Taj Mahal (Agra), Jaipur (Pink City), Goa (beaches), and Kerala (backwaters).",

    # South Africa
    "The ideal time to visit South Africa is during its spring (September to October) or autumn (April to May), offering mild weather and excellent wildlife viewing opportunities.",
    "South African foods include Biltong, Boerewors, Braai (BBQ), Potjiekos, Malva Pudding, and Bunny Chow.",
    "South African culture is known for its 'Rainbow Nation' diversity, rich history, traditional music, and vibrant arts scene.",
    "South Africa has 11 official languages, including Zulu, Xhosa, Afrikaans, and English. Its currency is the South African Rand (ZAR).",
    "Top universities in South Africa include the University of Cape Town and Wits University.",
    "Major tourist places in South Africa are Cape Town (Table Mountain, Robben Island), Kruger National Park, and the Garden Route.",

    # Australia
    "The best time to visit Australia varies greatly by region, but spring (September to November) and autumn (March to May) generally offer pleasant conditions nationwide.",
    "Iconic Australian foods are Vegemite, Lamingtons, Meat Pies, Barbecued seafood, Tim Tams, and Pavlova.",
    "Australian culture emphasizes a relaxed lifestyle, outdoor activities, sports, and a strong sense of mateship.",
    "The official language of Australia is English. Its currency is the Australian Dollar (AUD).",
    "Top universities in Australia include the University of Melbourne and the University of Sydney.",
    "Major tourist places in Australia are Sydney (Opera House, Harbour Bridge), Great Barrier Reef, Uluru (Ayers Rock), and Melbourne.",

    # Mexico
    "The best time to visit Mexico is during the dry season (December to April) to enjoy sunny days and comfortable temperatures.",
    "Popular Mexican foods include Tacos, Enchiladas, Guacamole, Chiles Rellenos, Tamales, and Mole.",
    "Mexican culture is a rich blend of indigenous traditions and Spanish influence, known for its vibrant festivals (like Day of the Dead), Mariachi music, and colorful folk art.",
    "The official language of Mexico is Spanish. Its currency is the Mexican Peso (MXN).",
    "Top universities in Mexico include the National Autonomous University of Mexico (UNAM) and Tecnológico de Monterrey.",
    "Major tourist places in Mexico are Mexico City (Teotihuacan, Zócalo), Cancún, Tulum, and Oaxaca.",

    # China
    "The best time to visit China is spring (April to May) and autumn (September to October) for pleasant weather and fewer national holidays.",
    "Chinese cuisine is diverse, with popular dishes like Peking Duck, Dim Sum, Kung Pao Chicken, Hot Pot, and Ma Po Tofu.",
    "Chinese culture is one of the world's oldest, known for its rich history, philosophy (Confucianism, Taoism), traditional arts (calligraphy, opera), and elaborate festivals.",
    "The official language of China is Mandarin Chinese. Its currency is the Chinese Yuan (CNY).",
    "Top universities in China include Tsinghua University and Peking University.",
    "Major tourist places in China are the Great Wall, Forbidden City (Beijing), Terracotta Army (Xi'an), and Shanghai's Bund.",

    # Spain
    "The best time to visit Spain is spring (March to May) and autumn (September to November) for mild weather, vibrant festivals, and fewer tourists.",
    "Famous Spanish foods include Paella, Tapas, Jamón Ibérico, Churros, Gazpacho, and Tortilla Española.",
    "Spanish culture is known for its passionate music and dance (Flamenco), bullfighting (though controversial), siestas, and lively festivals.",
    "The official language of Spain is Spanish. Its currency is the Euro (EUR).",
    "Top universities in Spain include the University of Barcelona and the Autonomous University of Madrid.",
    "Major tourist places in Spain are Barcelona (Sagrada Familia, Park Güell), Madrid (Prado Museum, Royal Palace), Seville (Alcázar), and the Alhambra (Granada).",

    # Germany
    "The best time to visit Germany is May to September for warm weather, outdoor activities, and many festivals like Oktoberfest.",
    "Traditional German foods are Bratwurst, Sauerkraut, Schnitzel, Pretzels, Black Forest Cake, and Königsberger Klopse.",
    "German culture is known for its efficiency, punctuality, engineering prowess, philosophical heritage, and a rich tradition of classical music and literature.",
    "The official language of Germany is German. Its currency is the Euro (EUR).",
    "Top universities in Germany include Technical University of Munich and Heidelberg University.",
    "Major tourist places in Germany are Berlin (Brandenburg Gate, Reichstag), Munich (Marienplatz), Neuschwanstein Castle, and the Rhine Valley."
]


# First, index the manual travel data
logging.info(f"Indexing {len(manual_travel_data)} manual travel fact chunks...")
for chunk in manual_travel_data:
    docs.append(chunk)
    emb = model.encode([chunk])
    index.add(np.array(emb))
logging.info("Manual travel data indexed successfully.")

# Then, index general facts from Wikipedia for each country
indexed_countries_from_wiki = 0
for country_name in countries_to_load:
    logging.info(f"Attempting to fetch general summary from Wikipedia for '{country_name}'...")
    # Removed time.sleep(0.5) here, as the robust function handles its own backoff/delay
    # and caching makes this less critical on subsequent runs.
    summary_text = fetch_wikipedia_summary_robust(country_name, sentences=3)

    if summary_text:
        # Split into sentences and filter out irrelevant content
        split_chunks = [chunk.strip() for chunk in summary_text.split(". ") if chunk.strip()]

        # Apply the new filter to Wikipedia chunks
        filtered_chunks = [chunk for chunk in split_chunks if filter_wiki_chunk(chunk)]

        if not filtered_chunks:
            logging.warning(f"No meaningful or relevant chunks extracted from Wikipedia summary for '{country_name}' after filtering. Skipping.")
            continue

        for chunk in filtered_chunks:
            docs.append(chunk)
            emb = model.encode([chunk])
            index.add(np.array(emb))
        indexed_countries_from_wiki += 1
        logging.info(f"Indexed {len(filtered_chunks)} relevant chunks from Wikipedia for '{country_name}'.")
    else:
        logging.warning(f"No Wikipedia summary obtained or indexed for '{country_name}'.")


logging.info(f"Finished indexing. Total countries with Wikipedia summaries indexed: {indexed_countries_from_wiki}/{len(countries_to_load)}")
logging.info(f"Total document chunks in knowledge base (manual + Wikipedia): {len(docs)}")
if len(docs) == 0:
    logging.error("FATAL ERROR: No documents were indexed. The chatbot will not function correctly. Please check data sources or network access.")
    import sys
    sys.exit(1)

# ✅ STEP 4: Load QA Pipeline
logging.info("Loading Question-Answering pipeline...")
try:
    qa_pipeline = pipeline("question-answering", model="deepset/roberta-base-squad2")
    logging.info("QA pipeline loaded successfully with deepset/roberta-base-squad2.")
except Exception as e:
    logging.error(f"FATAL ERROR: Could not load QA pipeline. Error: {e}")
    import sys
    sys.exit(1)

# Helper function to extract country name from query
def get_country_from_query(query_lower, keywords):
    for keyword in keywords:
        if keyword in query_lower:
            parts = query_lower.split(keyword)
            if len(parts) > 1:
                country = parts[1].strip().replace("?", "").replace("the ", "")
                # Simple cleanup for common trailing words
                for term in ["culture", "foods", "universities", "places", "time", "currency", "language", "in", "about", "to"]:
                    if country.endswith(term):
                        country = country[:-len(term)].strip()
                # Handle cases like "french culture"
                if "french" in country:
                    return "france"
                if "japanese" in country:
                    return "japan"
                if "italian" in country:
                    return "italy"
                if "canadian" in country:
                    return "canada"
                if "thai" in country:
                    return "thailand"
                if "indian" in country:
                    return "india"
                if "south african" in country:
                    return "south africa"
                if "australian" in country:
                    return "australia"
                if "mexican" in country:
                    return "mexico"
                if "chinese" in country:
                    return "china"
                # FIXED: Ensure "spa" doesn't become the country name for Spain
                if "spanish" in country or country.lower().startswith("spa"):
                    return "spain"
                if "german" in country:
                    return "germany"
                if "brazilian" in country:
                    return "brazil"
                if "moroccan" in country:
                    return "morocco"
                if "american" in country or "usa" in country or "united states" in country:
                    return "united states" # Ensuring consistency for USA

                return country
    return None

# ✅ STEP 5: RAG Answer Function
def rag_answer(user_query):
    logging.info(f"Received query: '{user_query}'")

    if not docs:
        return "I'm sorry, my knowledge base is empty. I cannot answer any questions at this time."

    q_emb = model.encode([user_query])

    k_retrieval = 7 # Keeping at 7 for now, as Roberta can handle more context
    D, I = index.search(np.array(q_emb), k=k_retrieval)

    retrieved_chunks = [docs[i] for i in I[0] if i < len(docs)]
    context = " ".join(retrieved_chunks)

    logging.info(f"Retrieved {len(retrieved_chunks)} chunks. Context length: {len(context)} characters.")
    # For debugging the retrieved context, uncomment the line below:
    print(f"\n--- Retrieved Context for '{user_query}' ---\n{context}\n---------------------------------------\n")

    if not context.strip() or len(context.split()) < 10: # Lowered minimum context words slightly
        logging.warning("Insufficient or irrelevant context retrieved for the query.")
        return "Sorry, I couldn't find enough relevant information to answer that question from my current knowledge base. My information is limited to general country data and common travel tips. Please try a different question or rephrase."

    try:
        result = qa_pipeline(question=user_query, context=context)
        answer = result.get("answer", "").strip() # Ensure answer is stripped
        score = result.get('score', 0.0)

        # Very low confidence threshold for *any* extraction to be considered for rephrasing
        min_qa_score_for_specific_answer = 0.01

        user_query_lower = user_query.lower()
        country = get_country_from_query(user_query_lower,
                                         ["currency of", "money in", "best time to visit", "when to visit",
                                          "what to eat in", "foods in", "dishes in", "try in",
                                          "top universities in", "universities in", "culture of",
                                          "tell me about", "about the culture in", "major tourist places in",
                                          "where to visit in", "tourist attractions in", "places to see in",
                                          "official language of", "languages of", "what language in",
                                          "french culture", "japanese culture", "italian culture", "canadian culture",
                                          "thai culture", "indian culture", "south african culture", "australian culture",
                                          "mexican culture", "chinese culture", "spanish culture", "german culture",
                                          "brazilian culture", "moroccan culture", "american culture"])


        # --- Specific Answer Re-phrasing for Confident Output ---
        # This section checks the query type and the extracted answer, then formats it confidently.

        # Currency questions
        if country and ("currency" in user_query_lower or "money" in user_query_lower):
            if "japan" in country and "japanese yen" in answer.lower():
                return f"The currency of Japan is the Japanese Yen (JPY)."
            if "canada" in country and "canadian dollar" in answer.lower():
                return f"The currency of Canada is the Canadian Dollar (CAD)."
            if "italy" in country and "euro" in answer.lower():
                 return f"The currency of Italy is the Euro (EUR)."
            if "france" in country and "euro" in answer.lower():
                 return f"The currency of France is the Euro (EUR)."
            if "united states" in country and ("dollar" in answer.lower() or "usd" in answer.lower()):
                 return f"The currency of the United States is the United States Dollar (USD)."
            if "brazil" in country and "brazilian real" in answer.lower():
                 return f"The currency of Brazil is the Brazilian Real (BRL)."
            if "morocco" in country and "moroccan dirham" in answer.lower():
                 return f"The currency of Morocco is the Moroccan Dirham (MAD)."
            if "thailand" in country and "thai baht" in answer.lower():
                 return f"The currency of Thailand is the Thai Baht (THB)."
            if "india" in country and "indian rupee" in answer.lower():
                 return f"The currency of India is the Indian Rupee (INR)."
            if "south africa" in country and "south african rand" in answer.lower():
                 return f"The currency of South Africa is the South African Rand (ZAR)."
            if "australia" in country and ("australian dollar" in answer.lower() or "aud" in answer.lower()):
                 return f"The currency of Australia is the Australian Dollar (AUD)."
            if "mexico" in country and "mexican peso" in answer.lower():
                 return f"The currency of Mexico is the Mexican Peso (MXN)."
            if "china" in country and "chinese yuan" in answer.lower():
                 return f"The currency of China is the Chinese Yuan (CNY)."
            if "spain" in country and "euro" in answer.lower():
                 return f"The currency of Spain is the Euro (EUR)."
            if "germany" in country and "euro" in answer.lower():
                 return f"The currency of Germany is the Euro (EUR)."
            # Generic fallback for currency if a country was detected but not a specific match
            if answer: return f"The currency of {country.capitalize()} is {answer}."

        # Best time to visit questions
        if country and ("best time to visit" in user_query_lower or "when to visit" in user_query_lower):
            if "japan" in country and ("spring" in answer.lower() or "autumn" in answer.lower() or "march to may" in answer.lower() or "september to november" in answer.lower()):
                return f"The best time to visit Japan is {answer}."
            if "italy" in country and ("spring" in answer.lower() or "fall" in answer.lower() or "april to may" in answer.lower() or "september to october" in answer.lower()):
                return f"The best time to visit Italy is usually {answer}."
            if "france" in country and ("spring" in answer.lower() or "autumn" in answer.lower() or "april to june" in answer.lower() or "september to november" in answer.lower()):
                return f"The best time to visit France is {answer}."
            if "south africa" in country and ("spring" in answer.lower() or "autumn" in answer.lower() or "september to october" in answer.lower() or "april to may" in answer.lower()):
                return f"The ideal time to visit South Africa is during its {answer}."
            if "india" in country and ("october to march" in answer.lower() or "cooler" in answer.lower() or "drier months" in answer.lower()):
                return f"The best time to visit India is generally during the {answer}."
            if "canada" in country and ("summer" in answer.lower() or "winter" in answer.lower() or "june to august" in answer.lower() or "december to march" in answer.lower()):
                return f"The best time to visit Canada is typically {answer}."
            if "australia" in country and ("spring" in answer.lower() or "autumn" in answer.lower() or "september to november" in answer.lower() or "march to may" in answer.lower()):
                return f"The best time to visit Australia is generally {answer}."
            if "mexico" in country and ("dry season" in answer.lower() or "december to april" in answer.lower()):
                return f"The best time to visit Mexico is during the {answer}."
            if "china" in country and ("spring" in answer.lower() or "autumn" in answer.lower() or "april to may" in answer.lower() or "september to october" in answer.lower()):
                return f"The best time to visit China is {answer}."
            # FIXED: Added a more specific check for USA
            if "united states" in country and ("spring" in answer.lower() or "fall" in answer.lower() or "summer" in answer.lower() or "winter" in answer.lower() or "april-may" in answer.lower() or "september-october" in answer.lower()):
                return f"The best time to visit the USA is {answer}."
            if "spain" in country and ("spring" in answer.lower() or "autumn" in answer.lower() or "march to may" in answer.lower() or "september to november" in answer.lower()):
                return f"The best time to visit Spain is {answer}."
            if "germany" in country and ("may to september" in answer.lower() or "warm weather" in answer.lower()):
                return f"The best time to visit Germany is {answer}."
            if answer: return f"The best time to visit {country.capitalize()} is {answer}."


        # What to eat questions
        if country and ("what to eat" in user_query_lower or "foods in" in user_query_lower or "dishes in" in user_query_lower or "try in" in user_query_lower):
            if "japan" in country and ("sushi" in answer.lower() or "ramen" in answer.lower() or "tempura" in answer.lower() or "udon" in answer.lower()):
                return f"When in Japan, you should try foods like {answer}."
            if "italy" in country and ("pizza" in answer.lower() or "pasta" in answer.lower() or "gelato" in answer.lower() or "tiramisu" in answer.lower()):
                return f"When in Italy, you should try foods like {answer}."
            if "france" in country and ("croissants" in answer.lower() or "baguette" in answer.lower() or "coq au vin" in answer.lower() or "macarons" in answer.lower()):
                return f"In France, you should try iconic foods such as {answer}."
            if "united states" in country and ("hamburgers" in answer.lower() or "hot dogs" in answer.lower() or "apple pie" in answer.lower()):
                return f"In the United States, you should try popular foods like {answer}."
            if "brazil" in country and ("feijoada" in answer.lower() or "pão de queijo" in answer.lower() or "churrasco" in answer.lower()):
                return f"In Brazil, you should try traditional foods like {answer}."
            if "morocco" in country and ("tagine" in answer.lower() or "couscous" in answer.lower() or "mint tea" in answer.lower()):
                return f"In Morocco, you should try foods like {answer}."
            if "canada" in country and ("poutine" in answer.lower() or "maple syrup" in answer.lower() or "nanaimo bars" in answer.lower()):
                return f"In Canada, you should try foods like {answer}."
            if "thailand" in country and ("pad thai" in answer.lower() or "tom yum goong" in answer.lower() or "green curry" in answer.lower()):
                return f"In Thailand, you should try famous dishes such as {answer}."
            if "india" in country and ("curry" in answer.lower() or "biryani" in answer.lower() or "naan" in answer.lower()):
                return f"In India, you should try diverse foods like {answer}."
            if "south africa" in country and ("biltong" in answer.lower() or "boerewors" in answer.lower() or "braai" in answer.lower()):
                return f"In South Africa, you should try foods like {answer}."
            if "australia" in country and ("vegemite" in answer.lower() or "lamingtons" in answer.lower() or "meat pies" in answer.lower()):
                return f"In Australia, you should try iconic foods such as {answer}."
            if "mexico" in country and ("tacos" in answer.lower() or "enchiladas" in answer.lower() or "guacamole" in answer.lower()):
                return f"In Mexico, you should try popular foods like {answer}."
            if "china" in country and ("peking duck" in answer.lower() or "dim sum" in answer.lower() or "hot pot" in answer.lower()):
                return f"In China, you should try dishes like {answer}."
            if "spain" in country and ("paella" in answer.lower() or "tapas" in answer.lower() or "churros" in answer.lower()):
                return f"In Spain, you should try famous foods like {answer}."
            if "germany" in country and ("bratwurst" in answer.lower() or "sauerkraut" in answer.lower() or "schnitzel" in answer.lower()):
                return f"In Germany, you should try traditional foods like {answer}."
            if answer: return f"When in {country.capitalize()}, you should try foods like {answer}."


        # Top universities questions
        if country and ("top universities" in user_query_lower or "universities in" in user_query_lower):
            if "japan" in country and ("university of tokyo" in answer.lower() or "kyoto university" in answer.lower()):
                return f"In Japan, top universities include {answer}."
            if "italy" in country and ("sapienza university of rome" in answer.lower() or "university of bologna" in answer.lower()):
                return f"In Italy, top universities include {answer}."
            if "france" in country and ("psl research university paris" in answer.lower() or "sorbonne university" in answer.lower()):
                return f"In France, top universities include {answer}."
            if "united states" in country and ("harvard university" in answer.lower() or "stanford university" in answer.lower() or "mit" in answer.lower()):
                return f"In the United States, top universities include {answer}."
            if "brazil" in country and ("university of são paulo" in answer.lower() or "university of campinas" in answer.lower()):
                return f"In Brazil, top universities include {answer}."
            if "morocco" in country and ("mohammed v university" in answer.lower() or "cadi ayyad university" in answer.lower()):
                return f"In Morocco, top universities include {answer}."
            if "canada" in country and ("university of toronto" in answer.lower() or "mcgill university" in answer.lower() or "university of british columbia" in answer.lower()):
                return f"In Canada, top universities include {answer}."
            if "thailand" in country and ("chulalongkorn university" in answer.lower() or "mahidol university" in answer.lower()):
                return f"In Thailand, top universities include {answer}."
            if "india" in country and ("indian institute of science" in answer.lower() or "iit delhi" in answer.lower()):
                return f"In India, top universities include {answer}."
            if "south africa" in country and ("university of cape town" in answer.lower() or "wits university" in answer.lower()):
                return f"In South Africa, top universities include {answer}."
            if "australia" in country and ("university of melbourne" in answer.lower() or "university of sydney" in answer.lower()):
                return f"In Australia, top universities include {answer}."
            if "mexico" in country and ("national autonomous university of mexico" in answer.lower() or "tecnológico de monterrey" in answer.lower()):
                return f"In Mexico, top universities include {answer}."
            if "china" in country and ("tsinghua university" in answer.lower() or "peking university" in answer.lower()):
                return f"In China, top universities include {answer}."
            if "spain" in country and ("university of barcelona" in answer.lower() or "autonomous university of madrid" in answer.lower()):
                return f"In Spain, top universities include {answer}."
            if "germany" in country and ("technical university of munich" in answer.lower() or "heidelberg university" in answer.lower()):
                return f"In Germany, top universities include {answer}."
            if answer: return f"Some top universities in {country.capitalize()} include {answer}."

        # Culture questions
        if country and ("culture of" in user_query_lower or "tell me about" in user_query_lower or "about the culture in" in user_query_lower or "culture" in user_query_lower):
            if ("france" in country or "french" in country) and ("elegance" in answer.lower() or "haute couture" in answer.lower() or "gastronomy" in answer.lower() or "art and philosophy" in answer.lower()):
                return f"French culture is celebrated for its {answer}."
            if "japan" in country and ("respect" in answer.lower() or "harmony" in answer.lower() or "punctuality" in answer.lower() or "tea ceremony" in answer.lower()):
                return f"Japanese culture emphasizes {answer}."
            if "italy" in country and ("art" in answer.lower() or "fashion" in answer.lower() or "family values" in answer.lower() or "opera" in answer.lower()):
                return f"Italian culture is known for its {answer}."
            if "united states" in country and ("diverse" in answer.lower() or "individualism" in answer.lower() or "innovation" in answer.lower() or "pop culture" in answer.lower()):
                return f"American culture is {answer}."
            if "brazil" in country and ("vibrant and diverse" in answer.lower() or "carnival" in answer.lower() or "samba music" in answer.lower() or "football passion" in answer.lower()):
                return f"Brazilian culture is {answer}."
            if "morocco" in country and ("blend of arab, berber, and european influences" in answer.lower() or "hospitality" in answer.lower() or "vibrant souks" in answer.lower()):
                return f"Moroccan culture is a {answer}."
            if "canada" in country and ("politeness" in answer.lower() or "multiculturalism" in answer.lower() or "natural beauty appreciation" in answer.lower()):
                return f"Canadian culture is known for its {answer}."
            if "thailand" in country and ("deeply influenced by buddhism" in answer.lower() or "respect" in answer.lower() or "hospitality" in answer.lower()):
                return f"Thai culture is {answer}."
            if "india" in country and ("rich in diversity" in answer.lower() or "ancient traditions" in answer.lower() or "spirituality" in answer.lower()):
                return f"Indian culture is {answer}."
            if "south africa" in country and ("rainbow nation" in answer.lower() or "diversity" in answer.lower() or "rich history" in answer.lower()):
                return f"South African culture is known for its {answer}."
            if "australia" in country and ("relaxed lifestyle" in answer.lower() or "outdoor activities" in answer.lower() or "sports" in answer.lower()):
                return f"Australian culture emphasizes a {answer}."
            if "mexico" in country and ("rich blend of indigenous traditions and spanish influence" in answer.lower() or "vibrant festivals" in answer.lower() or "mariachi music" in answer.lower()):
                return f"Mexican culture is a {answer}."
            if "china" in country and ("one of the world's oldest" in answer.lower() or "rich history" in answer.lower() or "philosophy" in answer.lower()):
                return f"Chinese culture is {answer}."
            if "spain" in country and ("passionate music and dance" in answer.lower() or "flamenco" in answer.lower() or "bullfighting" in answer.lower() or "lively festivals" in answer.lower()):
                return f"Spanish culture is known for its {answer}."
            if "germany" in country and ("efficiency" in answer.lower() or "punctuality" in answer.lower() or "engineering prowess" in answer.lower()):
                return f"German culture is known for its {answer}."
            if answer: return f"The culture of {country.capitalize()} is known for {answer}."


        # Major Tourist Places questions
        if country and ("major tourist places in" in user_query_lower or "where to visit in" in user_query_lower or "tourist attractions in" in user_query_lower or "places to see in" in user_query_lower):
            if "japan" in country and ("tokyo" in answer.lower() or "kyoto" in answer.lower() or "mount fuji" in answer.lower()):
                return f"In Japan, you could visit {answer}."
            if "italy" in country and ("rome" in answer.lower() or "florence" in answer.lower() or "venice" in answer.lower()):
                return f"In Italy, you could visit {answer}."
            if "france" in country and ("paris" in answer.lower() or "eiffel tower" in answer.lower() or "louvre museum" in answer.lower() or "french riviera" in answer.lower()):
                return f"In France, you could visit {answer}."
            if "united states" in country and ("new york city" in answer.lower() or "grand canyon" in answer.lower() or "florida's theme parks" in answer.lower()):
                return f"In the United States, you could visit {answer}."
            if "brazil" in country and ("rio de janeiro" in answer.lower() or "iguazu falls" in answer.lower() or "amazon rainforest" in answer.lower()):
                return f"In Brazil, you could visit {answer}."
            if "morocco" in country and ("marrakech" in answer.lower() or "fes" in answer.lower() or "chefchaouen" in answer.lower() or "sahara desert" in answer.lower()):
                return f"In Morocco, you could visit {answer}."
            if "canada" in country and ("banff national park" in answer.lower() or "vancouver" in answer.lower() or "toronto" in answer.lower() or "niagara falls" in answer.lower()):
                return f"In Canada, you could visit {answer}."
            if "thailand" in country and ("bangkok" in answer.lower() or "phuket" in answer.lower() or "chiang mai" in answer.lower()):
                return f"In Thailand, you could visit {answer}."
            if "india" in country and ("taj mahal" in answer.lower() or "jaipur" in answer.lower() or "goa" in answer.lower()):
                return f"In India, you could visit {answer}."
            if "south africa" in country and ("cape town" in answer.lower() or "kruger national park" in answer.lower() or "garden route" in answer.lower()):
                return f"In South Africa, you could visit {answer}."
            if "australia" in country and ("sydney" in answer.lower() or "great barrier reef" in answer.lower() or "uluru" in answer.lower()):
                return f"In Australia, you could visit {answer}."
            if "mexico" in country and ("mexico city" in answer.lower() or "cancún" in answer.lower() or "tulum" in answer.lower()):
                return f"In Mexico, you could visit {answer}."
            if "china" in country and ("great wall" in answer.lower() or "forbidden city" in answer.lower() or "terracotta army" in answer.lower()):
                return f"In China, you could visit {answer}."
            if "spain" in country and ("barcelona" in answer.lower() or "madrid" in answer.lower() or "sagrada familia" in answer.lower()):
                return f"In Spain, you could visit {answer}."
            if "germany" in country and ("berlin" in answer.lower() or "munich" in answer.lower() or "neuschwanstein castle" in answer.lower()):
                return f"In Germany, you could visit {answer}."
            if answer: return f"In {country.capitalize()}, you could visit {answer}."


        # Official Languages questions
        if country and ("official language of" in user_query_lower or "languages of" in user_query_lower or "what language in" in user_query_lower):
            if "japan" in country and "japanese" in answer.lower():
                return f"The official language of Japan is {answer}."
            if "canada" in country and ("english" in answer.lower() or "french" in answer.lower()):
                return f"The official languages of Canada are {answer}."
            if "italy" in country and "italian" in answer.lower():
                return f"The official language of Italy is {answer}."
            if "south africa" in country and ("zulu" in answer.lower() or "xhosa" in answer.lower() or "afrikaans" in answer.lower() or "english" in answer.lower() or "11 official languages" in answer.lower()):
                 return f"South Africa has {answer}."
            if "india" in country and ("hindi" in answer.lower() or "english" in answer.lower()):
                 return f"The official languages of India are {answer}."
            if "united states" in country and ("english" in answer.lower()):
                return f"The official language of the United States is {answer}."
            if "brazil" in country and ("portuguese" in answer.lower()):
                return f"The official language of Brazil is {answer}."
            if "morocco" in country and ("arabic" in answer.lower() or "berber" in answer.lower()):
                return f"The official languages of Morocco are {answer}."
            if "thailand" in country and ("thai" in answer.lower()):
                return f"The official language of Thailand is {answer}."
            if "australia" in country and ("english" in answer.lower()):
                return f"The official language of Australia is {answer}."
            if "mexico" in country and ("spanish" in answer.lower()):
                return f"The official language of Mexico is {answer}."
            if "china" in country and ("mandarin chinese" in answer.lower()):
                return f"The official language of China is {answer}."
            if "spain" in country and ("spanish" in answer.lower()):
                return f"The official language of Spain is {answer}."
            if "germany" in country and ("german" in answer.lower()):
                return f"The official language of Germany is {answer}."
            if answer: return f"The official language(s) of {country.capitalize()} is/are {answer}."

        # Fallback for answers that were extracted but didn't fit specific rephrasing
        if answer and score >= min_qa_score_for_specific_answer:
            logging.info(f"Answer generated: '{answer}' (Score: {score:.2f}) - No specific rephrasing template found, returning raw answer.")
            return answer

        # Fallback for low confidence or no answer at all
        logging.info(f"Insufficient confidence ({score:.2f}) or no specific answer for '{user_query}'.")
        return (f"I'm sorry, I couldn't confidently find a specific answer for that question from my knowledge base (score: {score:.2f}). "
                "My information is limited to general country data and common travel tips. Please try rephrasing, or ask about: "
                "best travel times, popular foods, main tourist places, culture, languages, currencies, or top universities for these countries.")

    except Exception as e:
        logging.error(f"Error during QA pipeline processing for query '{user_query}': {e}")
        return "An internal error occurred while trying to answer your question. Please try again."

# ✅ STEP 6: Gradio UI
logging.info("Setting up Gradio UI...")
demo = gr.Interface(
    fn=rag_answer,
    inputs=gr.Textbox(label="Ask about a country or travel destination..."),
    outputs=gr.Textbox(label="Answer"),
    title="🌍 RAG Travel Chatbot (QA Version)",
    description=(
        "Ask travel questions about the listed countries (e.g., Japan, Italy, France, USA, Brazil, etc.). "
        "Examples: 'What should I eat in Italy?', 'Best time to visit Japan?', 'What is the currency of Canada?', "
        "'Tell me about French culture.', 'Top universities in Germany?', 'Where to visit in South Africa?', 'What are the languages of India?'"
    )
)

logging.info("Launching Gradio demo...")
demo.launch(share=True)
logging.info("Gradio demo launched.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 61.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

2025-06-18 03:51:39,999 - INFO - No existing cache file found at wikipedia_summary_cache.json. A new cache will be created.
2025-06-18 03:51:40,000 - INFO - Initializing Sentence-BERT model and FAISS index...
2025-06-18 03:51:40,005 - INFO - Use pytorch device_name: cpu
2025-06-18 03:51:40,006 - INFO - Load pretrained SentenceTransformer: all-MiniLM-L6-v2
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

...

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()


2025-06-18 03:53:10,316 - INFO - HTTP Request: GET https://api.gradio.app/v3/tunnel-request "HTTP/1.1 200 OK"
2025-06-18 03:53:10,493 - INFO - HTTP Request: GET https://cdn-media.huggingface.co/frpc-gradio-0.3/frpc_linux_amd64 "HTTP/1.1 200 OK"


* Running on public URL: https://633451dad6cc60c2b0.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


2025-06-18 03:53:11,406 - INFO - HTTP Request: HEAD https://633451dad6cc60c2b0.gradio.live "HTTP/1.1 200 OK"


2025-06-18 03:53:11,411 - INFO - Gradio demo launched.
